# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice
import os
import shutil
import joblib 


In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

experiment_name = 'HyperDrive-Experiment'

experiment=Experiment(ws, experiment_name)

quick-starts-ws-136370
aml-quickstarts-136370
southcentralus
610d6e37-4747-4a20-80eb-3aad70a55f43


# Compute Cluster

In [3]:
cpu_cluster_name = "cpu-cluster"

# Verifying that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


## Dataset

The dataset is loaded in the supporting train.py script using URL


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

I chose Logisit Regression because it is a widely used classifier suitable for a binary classification problem.

The model will be trained using different combinations of C and max_iter hyperparameters. C is inverse of regularization strength. Like in support vector machines, smaller values specify stronger regularization. max_iter is the maximum number of iterations taken for the solvers to converge.  

I want to find the best C and max_iter combination to have a correct hyperparameter model that is trained with the right number of iterations it requires to converge and does not suffer from overfitting to show a good generalisation performance on the test data. 

I chose BanditPolicy as the early termination policy. According Microsoft documentation, Bandit policy is "based on slack criteria, and a frequency and delay interval for evaluation." Early termination is required for RandomParameterSampling. It does not allow us to try multiple combinations of the hyperparameters of the search space by reducing the amount of iterations/runs (a combination of the hyperparameters is tried during each iteration) that will not give better results than the ones that have already been completed. This helps save a lot of time and resources. Reference: https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive?view=azure-ml-py

In configuration settings, besides the estimator and BanditPolicy, I specified weighted under the curve  **AUC_weighted** as the primary metric to optimise because it handles unbalanced data. I defined the sampling method over the search space: RandomParameterSampling. This method chooses randomply the best hyperparameters to optimise for speed.



In [5]:
# An early termination policy. 
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)


# Different parameters that will be used during training
param_sampling = RandomParameterSampling( 
    {
    '--C': choice(0.0001, 0.001, 0.01, 0.1, 1,10,100,1000),
    '--max_iter': choice(100, 200, 300, 400, 500)
    }
)

if "training" not in os.listdir():
    os.mkdir("./training")

script_folder = "./training"    
    
# Reference: lesson 6.3: copying the training file into the script folder
shutil.copy('./train.py', script_folder)


# Estimator and hyperdrive config
estimator = SKLearn(source_directory = script_folder, 
              compute_target=cpu_cluster, 
              entry_script="train.py")

hyperdrive_run_config = HyperDriveConfig(estimator=estimator, 
                          hyperparameter_sampling=param_sampling,
                          policy=early_termination_policy, 
                          primary_metric_name = "AUC_weighted", 
                          primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                          max_total_runs = 15)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


## Submitting the Experiment

In [6]:
hyperdrive_run = experiment.submit(config = hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
#reference: lesson 6.3: running a history widget to show the progress
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output = True)
assert(hyperdrive_run.get_status() == "Completed")

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_63093e7c-170d-4d56-bcd0-2e8c85b8ebad
Web View: https://ml.azure.com/experiments/HyperDrive-Experiment/runs/HD_63093e7c-170d-4d56-bcd0-2e8c85b8ebad?wsid=/subscriptions/610d6e37-4747-4a20-80eb-3aad70a55f43/resourcegroups/aml-quickstarts-136370/workspaces/quick-starts-ws-136370

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-01-28T10:59:31.470717][API][INFO]Experiment created<END>\n""<START>[2021-01-28T10:59:32.032037][GENERATOR][INFO]Trying to sample '15' jobs from the hyperparameter space<END>\n""<START>[2021-01-28T10:59:32.293486][GENERATOR][INFO]Successfully sampled '15' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-01-28T10:59:33.1328315Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_63093e7c-170d-4d56-bcd0-2e8c85b8ebad
Web View: https://ml.azure.com/experiments/HyperDrive-Experiment/runs/HD_63093e7c-170d-4d56-bcd0-2e8c85b8ebad?wsid=/subsc

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [8]:
# Getting the best run and displaying its properties
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print(parameter_values)
print('Best Run Id: ', best_run.id)
print('AUC_weighted:', best_run_metrics['AUC_weighted'])


['--C', '0.1', '--max_iter', '400']
Best Run Id:  HD_63093e7c-170d-4d56-bcd0-2e8c85b8ebad_4
AUC_weighted: 0.8727777777777778


In [9]:
# Saving the best model
best_run.download_file("/outputs/model.joblib", "Logistic_Reg_Heart_Failure.joblib")

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service